In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv("train_Santander.csv",index_col = 0)
testdf = pd.read_csv("test_Santander.csv", index_col = 0)

df.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
ID,,,,,,,,,,,,,,,,,,,,,
1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [3]:
testdf.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
ID,,,,,,,,,,,,,,,,,,,,,
2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72
6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.95
7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.61
9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.73


In [3]:
y = df["TARGET"]
X = df.drop(['TARGET'], axis = 1)
X_test = testdf

Random Forest

In [4]:
std_scaler = StandardScaler().set_output(transform='pandas')
std_mm = MinMaxScaler()
rf = RandomForestClassifier()
prcomp = PCA().set_output(transform='pandas')

pipe = Pipeline([('SCL', std_mm), ('PCA',prcomp),('RF', rf)])


kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {'PCA__n_components': np.arange(6,12),'SCL':[std_scaler, std_mm, None], 'RF__max_depth':[2,3] }
gcv = GridSearchCV(pipe, param_grid = params, cv = kfold, scoring = 'roc_auc', refit=True)
gcv.fit(X, y)
print(gcv.best_score_)
print(gcv.best_params_)

y_pred_prob = gcv.predict_proba(X_test)
y_pred_prob = y_pred_prob[:,1]

submit = pd.DataFrame({'ID':list(testdf.index), 'TARGET': y_pred_prob})
submit

0.7743688203182628
{'PCA__n_components': 8, 'RF__max_depth': 3, 'SCL': MinMaxScaler()}


,ID,TARGET
0,2,0.021834
1,5,0.016285
2,6,0.017459
3,7,0.023651
4,9,0.012817
...,...,...
75813,151831,0.031666
75814,151832,0.018890
75815,151833,0.017380
75816,151834,0.024901


In [ ]:
submit.to_csv("Santander_RF_GCV.csv", index = False)

XGBoost with GCV

In [ ]:
#XGBoost

std_scaler = StandardScaler().set_output(transform='pandas')
std_mm = MinMaxScaler()
xgb=XGBClassifier(random_state=24)
prcomp = PCA().set_output(transform='pandas')

pipe = Pipeline([('SCL', std_mm), ('PCA',prcomp),('XGB', xgb)])
#pipe.fit(X, y)

kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {'PCA__n_components': np.arange(6,12),'SCL':[std_scaler, std_mm, None], 
          'XGB__max_depth':[2,3], 'XGB__learning_rate': np.linspace(0.001, 0.9, 10),'XGB__n_estimators': [25, 50, 100]}
gcv = GridSearchCV(pipe, param_grid = params, cv = kfold, scoring = 'roc_auc', refit=True)
gcv.fit(X, y)
print(gcv.best_score_)
print(gcv.best_params_)

y_pred_prob = gcv.predict_proba(X_test)
y_pred_prob = y_pred_prob[:,1]

submit = pd.DataFrame({'ID':list(testdf.index), 'TARGET': y_pred_prob})
submit

In [ ]:
submit.to_csv("Santander_XGB.csv", index = False)

CatBoost with GCV but no parameters for CatBoost

In [ ]:
#CatBoost

std_scaler = StandardScaler().set_output(transform='pandas')
std_mm = MinMaxScaler()
cb = CatBoostClassifier(random_state=24)
prcomp = PCA().set_output(transform='pandas')

pipe = Pipeline([('SCL', std_mm), ('PCA',prcomp),('CAT', cb)])
#pipe.fit(X, y)

kfold = KFold(n_splits = 5, shuffle = True, random_state=24)
params = {'PCA__n_components': np.arange(6,12),'SCL':[std_scaler, std_mm, None]}

gcv = GridSearchCV(pipe, param_grid = params, cv = kfold, scoring = 'roc_auc', refit=True)
gcv.fit(X, y)
print(gcv.best_score_)
print(gcv.best_params_)

y_pred_prob = gcv.predict_proba(X_test)
y_pred_prob = y_pred_prob[:,1]

submit = pd.DataFrame({'ID':list(testdf.index), 'TARGET': y_pred_prob})
submit

In [ ]:
submit.to_csv("Santander_CatBoost.csv", index = False)

LGBM without GCV

In [ ]:
#LGBM

std_scaler = StandardScaler().set_output(transform='pandas')
std_mm = MinMaxScaler()
lgb = LGBMClassifier(random_state=24)
prcomp = PCA().set_output(transform='pandas')

pipe = Pipeline([('SCL', std_mm), ('PCA',prcomp),('LGBM', lgb)])
pipe.fit(X, y)

y_pred_prob = pipe.predict_proba(X_test)
y_pred_prob = y_pred_prob[:,1]

submit = pd.DataFrame({'ID':list(testdf.index), 'TARGET': y_pred_prob})
submit

In [ ]:
submit.to_csv("Santander_LGBM.csv", index = False)